# Notebook para testear herramientas

Combinar carta de observacion con fchart

Astroplan para planear segun altura y masa de aire

reportlab para generar el pdf de observacion

In [ ]:
import fchart
import os

import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table
from astropy.table import Column
from astropy.io import ascii
from astroplan import FixedTarget

import conf as cf

In [ ]:
plots = cf.plots
if not os.path.isdir(plots):
    os.mkdir(plots)

In [ ]:
observatory = cf.observer

sunset_tonight = observatory.sun_set_time(cf.obs_time, which='next', horizon=-15*u.degree)
sunrise_tonight = observatory.sun_rise_time(cf.obs_time, which='next', horizon=-15*u.degree)

In [ ]:
print sunrise_tonight.iso
print sunset_tonight.iso

observatory.sun_rise_time

In [ ]:
white_cat= cf.catalog

white_table = ascii.read(white_cat, delimiter=' ', format='commented_header')

dist_lim = 80.
near = white_table['Dist'] < dist_lim 
visible = white_table['App_Mag']< 19     # Apparent Magnitude cut
bright = white_table['Abs_Mag']< -17.5      # Absolute Magnitude cut
lim_dec = white_table['Dec']< 30. 

sample = white_table[visible & bright & lim_dec & near]
sample

In [ ]:
targets = []
rises = np.empty(len(sample), dtype='str')
sets = np.empty(len(sample), dtype='str')
i = 0
for row in sample:
    coordinates = SkyCoord(row['RA']*u.degree, row['Dec']*u.degree, frame='icrs')
    obj = FixedTarget(coord=coordinates, name=row['Name'])
    targets.append(obj)
    rise_time = observatory.target_rise_time(cf.obs_time, obj)
    set_time = observatory.target_set_time(cf.obs_time, obj)
    if rise_time.jd != -999.0: rises[i] = rise_time.iso
    if set_time.jd != -999.0: sets[i] = set_time.iso
    i += 1

In [ ]:
targcol = Column(targets, name='planTargets')
risecol = Column(rises, name='RiseTime')
setcol = Column(sets, name='SetTime')

sample.add_columns(targcol)
sample.add_columns(risecol)
sample.add_columns(setcol)

In [ ]:
targets